In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install rouge 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install --upgrade pytorch-lightning


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 798 kB 4.9 MB/s 
     |████████████████████████████████| 125 kB 83.4 MB/s 
     |████████████████████████████████| 529 kB 71.0 MB/s 
     |████████████████████████████████| 87 kB 8.7 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=07db7d6c50e09590ecbdf8d932eecc5c6368822d5af13b5c4166fe7ae15c519f
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire


In [ ]:
import torch
torch.__version__

'1.12.1+cu113'

In [ ]:

!pip install gluonnlp==0.9.1 fasttext==0.9.2 pytorch_lightning==1.0.3 pybind11==2.5.0 pyrouge==0.1.3 easydict==1.9 gensim==3.8.3 konlpy==0.5.2 requests==2.24.0 setproctitle==1.1.10 tqdm==4.47.0 mxnet==1.6.0 unicode==2.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 252 kB 5.0 MB/s 
     |████████████████████████████████| 68 kB 8.4 MB/s 
     |████████████████████████████████| 533 kB 80.7 MB/s 
     |████████████████████████████████| 296 kB 87.5 MB/s 
     |████████████████████████████████| 60 kB 9.2 MB/s 
     |████████████████████████████████| 24.2 MB 772 kB/s 
     |████████████████████████████████| 19.4 MB 32.1 MB/s 
     |████████████████████████████████| 61 kB 516 kB/s 
     |████████████████████████████████| 66 kB 6.4 MB/s 
     |████████████████████████████████| 68.7 MB 1.2 MB/s 
     |████████████████████████████████| 829 kB 82.5 MB/s 
     |████████████████████████████████| 465 kB 73.3 MB/s 
     |████████████████████████████████| 86 kB 7.7 MB/s 
     |████████████████████████████████| 62 kB 1.9 MB/s 
     |████████████████████████████████| 62 kB 1.9 MB/s 
     |████████████████████████████████| 63 kB

In [ ]:
# !sudo apt-get install g++ openjdk-8-jdk python3-dev python3-pip curl
# !python3 -m pip install --upgrade pip
# !python3 -m pip install konlpy
# !sudo apt-get install curl git
# !bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

#Utils file which processes config.json

In [ ]:
import os
import json
from datetime import datetime

import easydict
import torch
from setproctitle import setproctitle


def config_parser(args=None, interpreter=None):
    if interpreter:
        with open(interpreter, 'rb') as f:
            config = dict(json.load(f))
        config = easydict.EasyDict({**config})
    else:
        with open(args['config_path'], 'rb') as f:
            config = dict(json.load(f))
        config = easydict.EasyDict({**config, **args})

    for k, v in args.items():
        config[k] = v

    return config


def write_output(test_loader, test_outputs, fname):
    save_dir, _ = os.path.split(fname)
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    header = ['id', 'src', 'tgt', 'pred']
    with open(fname, 'w') as f:
        print('\t'.join(header), file=f)
        for batch_idx, batch in enumerate(test_loader):
            top_k_outputs = test_outputs[batch_idx]['generated_summary']
            for k, output in enumerate(top_k_outputs, 1):
                data = [str(batch_idx) + '_' + str(k), batch.src_text[0], batch.tgt_text[0], output]
                print('\t'.join(data), file=f)
    print(f"Predicted outputs saved as {fname}")


def generate_exp_name(config):
    exp_name = datetime.now().strftime("%m-%d-%H:%M:%S")
    if len(config.exp_name):
        exp_name = exp_name + f'-{config.exp_name}'
    # set process title to exp name
    setproctitle(exp_name)
    print(f'Experiment results saved in logs/{exp_name}')
    return exp_name


def generate_output_name(config):
    if config.model_path is not None:
        basename = os.path.basename(config.model_path)
        model_name, _ = os.path.splitext(basename)
    else:
        model_name = 'best'
    if len(config.note):
        model_name += f'-{config.note}'
    basename = os.path.basename(config.path.test)
    filename, _ = os.path.splitext(basename)
    output_name = f'pred-{filename}-{model_name}.tsv'
    return output_name

In [ ]:
torch.cuda.is_available()

True

#Data processing

#Vocab File

In [ ]:
import json

PAD_TOKEN = '[PAD]'  # This has a vocab id, which is used to pad the encoder input, decoder input and target sequence
UNK_TOKEN = '[UNK]'  # This has a vocab id, which is used to represent out-of-vocabulary words
START_DECODING = '[START]'  # This has a vocab id, which is used at the start of every decoder input sequence
STOP_DECODING = '[STOP]'  # This has a vocab id, which is used at the end of untruncated target sequences


class Vocab(object):
    """
    Vocabulary class for mapping between words and ids (integers)
    """

    def __init__(self):
        self._word_to_id = {}
        self._id_to_word = []
        self._count = 0

    @classmethod
    def from_json(cls, vocab_file):
        vocab = cls()
        with open(vocab_file, 'r') as f:
            vocab._word_to_id = json.load(f)
        print(sorted(vocab._word_to_id,key=vocab._word_to_id.get,reverse=True))
        vocab._id_to_word = [w for w in sorted(vocab._word_to_id,
                                                    key=vocab._word_to_id.get,
                                                    reverse=True)]
        vocab._count = len(vocab._id_to_word)
        #vocab.specials = filter(vocab._id_to_word)
        return vocab

    @classmethod
    def from_counter(cls, counter, vocab_size, specials, min_freq):
        vocab = cls()
        word_and_freq = sorted(counter.items(), key=lambda tup: tup[0])
        word_and_freq.sort(key=lambda tup: tup[1], reverse=True)

        for w in specials:
            vocab._word_to_id[w] = vocab._count
            vocab._id_to_word += [w]
            vocab._count += 1

        for word, freq in word_and_freq:
            if freq < min_freq or vocab._count == vocab_size:
                break
            vocab._word_to_id[word] = vocab._count
            vocab._id_to_word += [word]
            vocab._count += 1
        return vocab

    def save(self, fpath):
        if not os.path.exists(os.path.dirname(fpath)):
            os.makedirs(os.path.dirname(fpath))
        with open(fpath, 'w', encoding='utf-8') as f:
            json.dump(self._word_to_id, f, ensure_ascii=False, indent=4)
        print(f'vocab file saved as {fpath}')

    def __len__(self):
        """Returns size of the vocabulary."""
        return self._count

    def word2id(self, word):
        """Returns the id (integer) of a word (string). Returns [UNK] id if word is OOV."""
        unk_id = self.unk()
        return self._word_to_id.get(word, unk_id)

    def id2word(self, word_id):
        """Returns the word (string) corresponding to an id (integer)."""
        if word_id not in self._id_to_word:
            raise ValueError(f'Id not found in vocab: {word_id}')
        return self._id_to_word[word_id]

    def size(self):
        """Returns the total size of the vocabulary."""
        return self._count

    def pad(self):
        """Helper to get index of pad symbol"""
        return self._word_to_id[PAD_TOKEN]

    def unk(self):
        """Helper to get index of unk symbol"""
        return self._word_to_id[UNK_TOKEN]

    def start(self):
        return self._word_to_id[START_DECODING]

    def stop(self):
        return self._word_to_id[STOP_DECODING]

    def extend(self, oovs):
        extended_vocab = self._id_to_word + list(oovs)
        return extended_vocab

    def tokens2ids(self, tokens):
        ids = [self.word2id(t) for t in tokens]
        return ids

    def source2ids_ext(self, src_tokens):
        """Maps source tokens to ids if in vocab, extended vocab ids if oov.
        Args:
            src_tokens: list of source text tokens
        Returns:
            ids: list of source text token ids
            oovs: list of oovs in source text
        """
        ids = []
        oovs = []
        for t in src_tokens:
            t_id = self.word2id(t)
            unk_id = self.word2id(UNK_TOKEN)
            if t_id == unk_id:
                if t not in oovs:
                    oovs.append(t)
                ids.append(self.size() + oovs.index(t))
            else:
                ids.append(t_id)
        return ids, oovs

    def target2ids_ext(self, tgt_tokens, oovs):
        """Maps target text to ids, using extended vocab (vocab + oovs).
        Args:
            tgt_tokens: list of target text tokens
            oovs: list of oovs from source text (copy mechanism)
        Returns:
            ids: list of target text token ids
        """
        ids = []
        for t in tgt_tokens:
            t_id = self.word2id(t)
            unk_id = self.word2id(UNK_TOKEN)
            if t_id == unk_id:
                if t in oovs:
                    ids.append(self.size() + oovs.index(t))
                else:
                    ids.append(unk_id)
            else:
                ids.append(t_id)
        return ids

    def outputids2words(self, ids, src_oovs):
        """Maps output ids to words
        Args:
            ids: list of ids
            src_oovs: list of oov words
        Returns:
            words: list of words mapped from ids
        """
        words = []
        extended_vocab = self.extend(src_oovs)
        for i in ids:
            try:
                w = self.id2word(i)  # might be oov
            except ValueError as e:
                assert src_oovs is not None, "Error: model produced a word ID that isn't in the vocabulary."
                try:
                    w = extended_vocab[i]
                except IndexError as e:
                    raise ValueError(f'Error: model produced word ID {i} \
                                       but this example only has {len(src_oovs)} article OOVs')
            words.append(w)
        return words

## Utils file

In [ ]:
import os
import pickle
import torch
import random
import json
import pandas as pd
from tqdm import tqdm


def load_dataset(data_dir):
    print(f"Loading dataset from {data_dir}")
    if data_dir.endswith('.json'):
        src, tgt = load_json(data_dir)
    elif data_dir.endswith('.txt'):
        with open(data_dir, 'r') as f:
            data = [l.split('\t') for l in f.readlines()]
        src, tgt = list(zip(*data))
    elif data_dir.endswith('.csv'):
        src, tgt = load_csv(data_dir)
    else:
        src, tgt = None, None
    return src, tgt


def split_pkl(data_path):
    data_dir = os.path.dirname(data_path)
    with open(data_path, 'rb') as f:
        data = pickle.load(f)
    news_keys = list(data.keys())
    random.shuffle(news_keys)

    train_keys = news_keys[150:]
    dev_keys = news_keys[:100]
    test_keys = news_keys[100:150]

    train_dict = {k: data[k] for k in train_keys}
    dev_dict = {k: data[k] for k in dev_keys}
    test_dict = {k: data[k] for k in test_keys}

    data_paths = [os.path.join(data_dir, f'nikl_{d}.pkl') for d in ['train', 'dev', 'test']]
    data_dicts = [train_dict, dev_dict, test_dict]

    for p, d in zip(*(data_paths, data_dicts)):
        with open(p, 'wb') as f:
            pickle.dump(d, f)
            print(f'File saved as {p}')


def load_json(data_dir):
    import json
    with open(data_dir, 'r', encoding='utf-8') as f:
        data = json.load(f)
    src = [ex['content'] for ex in data]
    tgt = [ex['bot_summary'] for ex in data]
    return src, tgt

## Make changes here
def load_csv(data_dir):
    df = pd.read_csv(data_dir, encoding='utf-8')
    src = list(df['text'].values) 
    tgt = list(df['summary'].values)

    # sep = '\t' if data_dir.endswith('.tsv') else ','
    # import pandas as pd
    # try:
    #     df = pd.read_csv(data_dir, sep=sep,
    #                      header=0, encoding='utf-8')
    # except:
    #     try:
    #         sep = '\t'
    #         df = pd.read_csv(data_dir, sep=sep,
    #                          header=0, encoding='utf-8')
    #     except UnicodeDecodeError:
    #         df = pd.read_csv(data_dir, sep=',',
    #                          header=0, encoding='ISO-8859-1')
    # print(df.head())
    # if 'abstractive' in df.columns:
    #     src = list(df['contents'].values)
    #     tgt = list(df['abstractive'].values)

    # elif 'bot_summary' in df.columns:
    #     df['content'] = df['content'].astype(str)
    #     df['bot_summary'] = df['bot_summary'].astype(str)

    #     src = list(df['content'].values)
    #     tgt = list(df['bot_summary'].values)

    # else:
    #     raise IndexError
    return src, tgt


def collate_tokens(values, pad_idx, left_pad=False,
                   pad_to_length=None):
    # Simplified version of `collate_tokens` from fairseq.data.data_utils
    """Convert a list of 1d tensors into a padded 2d tensor."""
    values = list(map(torch.LongTensor, values))
    size = max(v.size(0) for v in values)
    size = size if pad_to_length is None else max(size, pad_to_length)
    res = values[0].new(len(values), size).fill_(pad_idx)

    def copy_tensor(src, dst):
        assert dst.numel() == src.numel()
        dst.copy_(src)

    for i, v in enumerate(values):
        copy_tensor(v, res[i][size - len(v):] if left_pad else res[i][:len(v)])
    return res

## Loss File

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Loss(nn.Module):
    """
    Computes nll loss (Eq. (6)), coverage loss (Eq. (12)),
    and the composite loss function that combines the two (Eq. (13)).
    """
    def __init__(self, args):
        super().__init__()
        self.use_coverage = args.use_coverage
        self.cov_weight = args.cov_weight   # hyperparameter lambda in Eq. (13)
        self.pad_id = args.pad_id

    def nll_loss(self, output, target):
        """
        Negative log likelihood of target word - Eq. (6)
        Args:
            output: predicted probs from each timestep      [B x V_x T]
            target: answer ids using extended vocab         [B x T]
        Returns:
            loss: nll loss value; averaged over batch & timestep
        """
        output = torch.log(output)
        loss = F.nll_loss(output, target,
                          ignore_index=self.pad_id,
                          reduction='mean')
        return loss

    def cov_loss(self, attn_dist, coverage, dec_pad_mask, dec_len):
        """
        Coverage loss at timestep t - Eq. (12)
        Args:
            attn_dist: attention distribution from all timesteps            [B x L x T]
            coverage: sum of previous attn dist's from all timesteps        [B x L x T]
            dec_pad_mask: target sequence padding masks [PAD] -> True       [B x T]
            dec_len: target sequence lengths                                [B]
        Returns:
            loss: coverage loss value; averaged over batch & timestep
        """
        min_val = torch.min(attn_dist, coverage)    # [B x L x T]
        loss = torch.sum(min_val, dim=1)            # [B x T]

        # ignore loss from [PAD] tokens
        loss = loss.masked_fill_(
            dec_pad_mask,
            0.0
        )
        avg_loss = torch.sum(loss) / torch.sum(dec_len)
        return avg_loss

    def forward(self, output, batch):
        """
        Eq. (13) - Composite loss
        Args:
            output: a dictionary of model outputs with the following keys
                - final_dist
                - attn_dist
                - coverage
            batch: `Batch` instance
        Returns:
            loss: final composite loss value
        """
        final_dist = output['final_dist']
        dec_target = batch.dec_target
        nll_loss = self.nll_loss(output=final_dist, target=dec_target)

        attn_dist = output['attn_dist']
        coverage = output['coverage']
        dec_pad_mask = batch.dec_pad_mask
        dec_len = batch.dec_len
        cov_loss = self.cov_loss(attn_dist, coverage, dec_pad_mask, dec_len)
        return nll_loss, cov_loss


def build_criterion(config):
    criterion = Loss(args=config.loss.args)
    return criterion


## Data File 

In [ ]:
from collections import Counter

# import gluonnlp as nlp
import torch
# from konlpy.tag import Mecab
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

class TextDataset(Dataset):
    """
    Args:
        txt: list of text samples
        max_len: max sequence length
    """

    def __init__(self, txt, max_len):
        # self.tokenizer = self.load_tokenizer()
        self.dataset = self.build_dataset(txt)
        self.max_len = max_len
        self.vocab = None

    def __getitem__(self, index):
        txt = self.dataset[index]
        tokens = word_tokenize(txt)
        tokens = self.truncate(tokens)
        length = len(tokens)
        return tokens, length

    def __len__(self):
        return len(self.dataset)

    def build_vocab(self, vocab_size, min_freq, specials):
        counter = Counter()
        for t in self.dataset:
            tokens = self.tokenize(t)
            counter.update(tokens)
        vocab = Vocab.from_counter(counter=counter,
                                   vocab_size=vocab_size,
                                   min_freq=min_freq,
                                   specials=specials)
        return vocab

    def build_dataset(self, txt):
        txt = list(map(self.preprocess, txt))
        return txt

    # def load_tokenizer(self):
    #     tokenizer = word_tokenize()
    #     return tokenizer

    def preprocess(self, text):
        # TODO: add preprocessing functions relevant to korean & our dataset
        text = text.lower()
        return text

    def truncate(self, tokens):
        if len(tokens) > self.max_len:
            return tokens[:self.max_len]
        else:
            return tokens

    def tokenize(self, text):
        """Converts text string to list of tokens"""
        tokens = word_tokenize(text)
        return tokens


class SummDataset(Dataset):
    """
    Args:
        src: (TextDataset) source dataset
        tgt: (TextDataset) target dataset
    """

    def __init__(self, vocab, src, tgt=None):
        if tgt is not None:
            assert len(src) == len(tgt), "Source and target must contain the same number of examples"
        self.src, self.tgt = src, tgt
        self.vocab = vocab

    def __getitem__(self, index):
        src, src_len = self.src[index]
        if self.tgt is not None:
            tgt, tgt_len = self.tgt[index]
        else:
            tgt, tgt_len = None, None
        return src, src_len, tgt, tgt_len

    def __len__(self):
        """Returns size of the dataset"""
        return len(self.src)


class Batch:
    def __init__(self, data, vocab, max_decode):
        src, src_len, tgt, tgt_len = list(zip(*data))
        self.vocab = vocab
        self.pad_id = self.vocab.pad()
        self.max_decode = max_decode

        # Encoder info
        self.enc_input, self.enc_len, self.enc_pad_mask = None, None, None
        # Additional info for pointer-generator network
        self.enc_input_ext, self.max_oov_len, self.src_oovs = None, None, None
        # Decoder info
        self.dec_input, self.dec_target, self.dec_len, self.dec_pad_mask = None, None, None, None

        # Build batch inputs
        self.init_encoder_seq(src, src_len)
        self.init_decoder_seq(tgt, tgt_len)

        # Save original strings
        self.src_text = src
        self.tgt_text = tgt

    def init_encoder_seq(self, src, src_len):
        src_ids = [self.vocab.tokens2ids(s) for s in src]

        self.enc_input = collate_tokens(values=src_ids,
                                        pad_idx=self.pad_id)
        self.enc_len = torch.LongTensor(src_len)
        self.enc_pad_mask = (self.enc_input == self.pad_id)

        # Save additional info for pointer-generator
        # Determine max number of source text OOVs in this batch
        src_ids_ext, oovs = zip(*[self.vocab.source2ids_ext(s) for s in src])
        # Store the version of the encoder batch that uses article OOV ids
        self.enc_input_ext = collate_tokens(values=src_ids_ext,
                                            pad_idx=self.pad_id)
        self.max_oov_len = max([len(oov) for oov in oovs])
        # Store source text OOVs themselves
        self.src_oovs = oovs

    def init_decoder_seq(self, tgt, tgt_len):
        tgt_ids = [self.vocab.tokens2ids(t) for t in tgt]
        tgt_ids_ext = [self.vocab.target2ids_ext(t, oov) for t, oov in zip(tgt, self.src_oovs)]

        # create decoder inputs
        dec_input, _ = zip(*[self.get_decoder_input_target(t, self.max_decode) for t in tgt_ids])

        self.dec_input = collate_tokens(values=dec_input,
                                        pad_idx=self.pad_id,
                                        pad_to_length=self.max_decode)

        # create decoder targets using extended vocab
        _, dec_target = zip(*[self.get_decoder_input_target(t, self.max_decode) for t in tgt_ids_ext])

        self.dec_target = collate_tokens(values=dec_target,
                                         pad_idx=self.pad_id,
                                         pad_to_length=self.max_decode)

        self.dec_len = torch.LongTensor(tgt_len)
        self.dec_pad_mask = (self.dec_input == self.pad_id)

    def get_decoder_input_target(self, tgt, max_len):
        dec_input = [self.vocab.start()] + tgt
        dec_target = tgt + [self.vocab.stop()]
        # truncate inputs longer than max length
        if len(dec_input) > max_len:
            dec_input = dec_input[:max_len]
            dec_target = dec_target[:max_len]
        assert len(dec_input) == len(dec_target)
        return dec_input, dec_target

    def __len__(self):
        return self.enc_input.size(0)

    def __str__(self):
        batch_info = {
            'src_text': self.src_text,
            'tgt_text': self.tgt_text,
            'enc_input': self.enc_input,  # [B x L]
            'enc_input_ext': self.enc_input_ext,  # [B x L]
            'enc_len': self.enc_len,  # [B]
            'enc_pad_mask': self.enc_pad_mask,  # [B x L]
            'src_oovs': self.src_oovs,  # list of length B
            'max_oov_len': self.max_oov_len,  # single int value
            'dec_input': self.dec_input,  # [B x T]
            'dec_target': self.dec_target,  # [B x T]
            'dec_len': self.dec_len,  # [B]
            'dec_pad_mask': self.dec_pad_mask,  # [B x T]
        }
        return str(batch_info)

    def to(self, device):
        self.enc_input = self.enc_input.to(device)
        self.enc_input_ext = self.enc_input_ext.to(device)
        self.enc_len = self.enc_len.to(device)
        self.enc_pad_mask = self.enc_pad_mask.to(device)
        self.dec_input = self.dec_input.to(device)
        self.dec_target = self.dec_target.to(device)
        self.dec_len = self.dec_len.to(device)
        self.dec_pad_mask = self.dec_pad_mask.to(device)
        return self


def build_dataset(data_path, config, is_train, vocab=None, load_vocab=None):
    args = config.data
    if is_train:
        src_txt, tgt_txt = load_dataset(data_path)
        src_train = TextDataset(src_txt, args.src_max_train)
        tgt_train = TextDataset(tgt_txt, args.tgt_max_train)
        if load_vocab is not None:
            vocab = Vocab.from_json(load_vocab)
        else:
            vocab = src_train.build_vocab(vocab_size=args.vocab_size,
                                          min_freq=args.vocab_min_freq,
                                          specials=[PAD_TOKEN,
                                                    UNK_TOKEN,
                                                    START_DECODING,
                                                    STOP_DECODING])
        dataset = SummDataset(src=src_train,
                              tgt=tgt_train,
                              vocab=vocab)
        return dataset, vocab

    else:
        assert vocab is not None
        src_txt, tgt_txt = load_dataset(data_path)
        src_test = TextDataset(src_txt, args.src_max_test)
        tgt_test = TextDataset(tgt_txt, args.tgt_max_test)
        dataset = SummDataset(src=src_test,
                              tgt=tgt_test,
                              vocab=vocab)
        return dataset


def build_dataloader(dataset, vocab, batch_size, max_decode, is_train, num_workers):
    shuffle = True if is_train else False
    data_loader = DataLoader(dataset,
                             batch_size=batch_size,
                             shuffle=shuffle,
                             collate_fn=lambda data, v=vocab, t=max_decode: Batch(data=data,
                                                                                  vocab=v,
                                                                                  max_decode=t),
                             num_workers=num_workers)
    return data_loader

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#collate_fn=lambda data, v=vocab, t=max_decode: Batch(data=data,vocab=v,max_decode=t)

In [ ]:
torch.__version__

'1.12.1+cu113'

#MODEL

#Decode File

In [ ]:
import re


class Hypothesis(object):
    def __init__(self, tokens, log_probs, hidden_state, cell_state, coverage):
        self.tokens = tokens
        self.log_probs = log_probs
        self.hidden_state = hidden_state
        self.cell_state = cell_state
        self.coverage = coverage

    def extend(self, token, log_prob, hidden_state, cell_state, coverage):
        return Hypothesis(tokens=self.tokens + [token],
                          log_probs=self.log_probs + [log_prob],
                          hidden_state=hidden_state,
                          cell_state=cell_state,
                          coverage=coverage)

    @property
    def latest_token(self):
        return self.tokens[-1]

    @property
    def avg_log_prob(self):
        return sum(self.log_probs) / len(self.tokens)


def postprocess(tokens,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True):
    if skip_special_tokens:
        tokens = [t for t in tokens if not is_special(t)]
    out_string = ' '.join(tokens)
    if clean_up_tokenization_spaces:
        out_string = clean_up_tokenization(out_string)
    return out_string


def is_special(token):
    res = re.search("\[[A-Z]+\]", token)
    if res is None:
        return False
    return token == res.group()


def clean_up_tokenization(out_string):
    """
    Reference : transformers.tokenization_utils_base
    Clean up a list of simple English tokenization artifacts like spaces before punctuations and abbreviated forms.
    Args:
        out_string (:obj:`str`): The text to clean up.
    Returns:
        :obj:`str`: The cleaned-up string.
    """
    out_string = (
        out_string.replace(" .", ".")
        .replace(" ?", "?")
        .replace(" !", "!")
        .replace(" ,", ",")
        .replace(" ' ", "'")
        .replace(" n't", "n't")
        .replace(" 'm", "'m")
        .replace(" 's", "'s")
        .replace(" 've", "'ve")
        .replace(" 're", "'re")
    )
    return out_string

#Importing requirements

In [ ]:
!pip install --upgrade pytorch-lightning


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pytorch_lightning-1.8.3.post1-py3-none-any.whl (798 kB)
     |████████████████████████████████| 78 kB 3.8 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.47.0
    Uninstalling tqdm-4.47.0:
      Successfully uninstalled tqdm-4.47.0
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 1.0.3
    Uninstalling pytorch-lightning-1.0.3:
      Successfully uninstalled pytorch-lightning-1.0.3


In [ ]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence
from torch.optim import Adagrad

from rouge import Rouge

Models

:   1) Encoder

:   2) Attention module

:   3) Decoder w/ attention

:   4) Pointer-generator network

#Encoder

In [ ]:
class Encoder(nn.Module):
    """
    Single-layer bidirectional LSTM
    B : batch size
    E : embedding size
    H : encoder hidden state dimension
    L : sequence length
    """

    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim,
                            num_layers=1, bidirectional=True, batch_first=True)
        self.reduce_h = nn.Linear(hidden_dim * 2, hidden_dim, bias=True)
        self.reduce_c = nn.Linear(hidden_dim * 2, hidden_dim, bias=True)

    def forward(self, src, src_lens):
        """
        Args:
            src: source token embeddings    [B x L x E]
            src_lens: source text length    [B]
        Returns:
            enc_hidden: sequence of encoder hidden states                  [B x L x 2H]
            (final_h, final_c): Tuple for decoder state initialization     [B x L x H]
        """

        # Pack the sequence into a PackedSequence object to feed to the LSTM.
        x = pack_padded_sequence(src, src_lens.to('cpu'), batch_first=True, enforce_sorted=False)

        # Get outputs from the LSTM
        output, (h, c) = self.lstm(x)  # [B x L x 2H], [2 x B x H], [2 x B x H]
        enc_hidden, _ = pad_packed_sequence(output, batch_first=True)

        # Concatenate bidirectional lstm states
        h = torch.cat((h[0], h[1]), dim=-1)  # [B x 2H]
        c = torch.cat((c[0], c[1]), dim=-1)  # [B x 2H]

        # Project to decoder hidden state size
        final_hidden = torch.relu(self.reduce_h(h))  # [B x H]
        final_cell = torch.relu(self.reduce_c(c))  # [B x H]

        return enc_hidden, (final_hidden, final_cell)

#Additive Attention

In [ ]:
class Attention(nn.Module):
    """
    Attention mechanism based on Bahdanau et al. (2015) - Eq. (1)(2)
    augmented with Coverage mechanism - Eq. (11)
    B : batch size
    L : source text length
    H : encoder hidden state dimension
    """

    def __init__(self, hidden_dim, use_coverage):
        super().__init__()
        # Eq. (1)
        self.v = nn.Linear(hidden_dim * 2, 1, bias=False)                       # v
        self.enc_proj = nn.Linear(hidden_dim * 2, hidden_dim * 2, bias=False)   # W_h
        self.dec_proj = nn.Linear(hidden_dim, hidden_dim * 2, bias=True)        # W_s, b_attn

        self.use_coverage = use_coverage
        if self.use_coverage:
            # Additional parameter for coverage vector; w_c in Eq. (11)
            self.w_c = nn.Linear(1, hidden_dim * 2, bias=False)

    def forward(self, dec_input, coverage, enc_hidden, enc_pad_mask):
        """
        Args:
            dec_input: decoder hidden state             [B x H]
            coverage: coverage vector                   [B x L]
            enc_hidden: encoder hidden states           [B x L x 2H]
            enc_pad_mask: encoder padding masks         [B x L]
        Returns:
            attn_dist: attention dist'n over src tokens [B x L]
        """

        # Eq. (1)
        enc_feature = self.enc_proj(enc_hidden)         # [B x L x 2H]
        dec_feature = self.dec_proj(dec_input)          # [B x 2H]
        dec_feature = dec_feature.unsqueeze(1)          # [B x 1 x 2H]
        scores = enc_feature + dec_feature              # [B x L x 2H]

        if self.use_coverage:
            # Eq. (11)
            coverage = coverage.unsqueeze(-1)           # [B x L x 1]
            cov_feature = self.w_c(coverage)            # [B x L x 2H]
            scores = scores + cov_feature

        scores = torch.tanh(scores)                     # [B x L x 2H]
        scores = self.v(scores)                         # [B x L x 1]
        scores = scores.squeeze(-1)                     # [B x L]

        # Don't attend over padding; fill '-inf' where enc_pad_mask == True
        if enc_pad_mask is not None:
            scores = scores.float().masked_fill_(
                enc_pad_mask,
                float('-inf')
            ).type_as(scores)  # FP16 support: cast to float and back

        # Eq. (2)
        attn_dist = F.softmax(scores, dim=-1)               # [B x L]

        return attn_dist

#decoder with attention

In [ ]:
class AttnDecoder(nn.Module):
    """
    Single-layer unidirectional LSTM with attention for a single timestep - Eq. (3)(4)
    B : batch size
    E : embedding size
    H : decoder hidden state dimension
    V : vocab size
    """

    def __init__(self, input_dim, hidden_dim, vocab_size, use_coverage):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTMCell(input_size=input_dim, hidden_size=hidden_dim)
        self.attention = Attention(hidden_dim, use_coverage)
        # Eq. (4)
        self.v = nn.Linear(hidden_dim * 3, hidden_dim, bias=True)   # V, b
        self.v_out = nn.Linear(hidden_dim, vocab_size, bias=True)   # V', b'

    def forward(self, dec_input, prev_h, prev_c, enc_hidden, enc_pad_mask, coverage):
        """
        Args:
            dec_input: decoder input embedding at timestep t    [B x E]
            prev_h: decoder hidden state from prev timestep     [B x H]
            prev_c: decoder cell state from prev timestep       [B x H]
            enc_hidden: encoder hidden states                   [B x L x 2H]
            enc_pad_mask: encoder masks for attn computation    [B x L]
            coverage: coverage vector at timestep t - Eq. (10)  [B x L]
        Returns:
            vocab_dist: predicted vocab dist'n at timestep t    [B x V]
            attn_dist: attention dist'n at timestep t           [B x L]
            context_vec: context vector at timestep t           [B x 2H]
            hidden: hidden state at timestep t                  [B x H]
            cell: cell state at timestep t                      [B x H]
        """

        # Get this step's decoder hidden state
        hidden, cell = self.lstm(dec_input, (prev_h, prev_c))   # [B x H], [B x H]

        # Compute attention distribution over enc states
        attn_dist = self.attention(dec_input=hidden,
                                   coverage=coverage,
                                   enc_hidden=enc_hidden,
                                   enc_pad_mask=enc_pad_mask)   # [B x L]

        # Eq. (3) - Sum weighted enc hidden states to make context vector
        # The context vector is used later to compute generation probability
        context_vec = torch.bmm(attn_dist.unsqueeze(1), enc_hidden)     # [B x 1 x 2H]
        context_vec = torch.sum(context_vec, dim=1)                     # [B x 2H]

        # Eq. (4)
        output = self.v(torch.cat([hidden, context_vec], dim=-1))       # [B x 3H] -> [B x H]
        output = self.v_out(output)                                     # [B x V]
        vocab_dist = F.softmax(output, dim=-1)                          # [B x V]
        return vocab_dist, attn_dist, context_vec, hidden, cell


#Pointer Generator network

In [ ]:
#object of this model has been made in the summarizationModel class
class PointerGenerator(nn.Module):
    """
        2.2. Pointer-generator network
        - Computes generation probability p_gen             - Eq. (8)
        - Computes prob dist'n over extended vocabulary     - Eq. (9)
        2.3. Coverage mechanism
        - Computes coverage vector for coverage loss        - Eq. (10)
        B : batch size
        E : decoder embedding size
        H : encoder, decoder hidden state dimension
        L : source text length
        V : vocab size
        V_x : extended vocab size
    """

    def __init__(self, config, vocab):
        super().__init__()
        self.config = config
        self.use_pretrained = config.model.use_pretrained
        self.vocab = vocab

        if self.use_pretrained:
            emb_vecs = self.load_embeddings(config.model.pretrained)
            self.embedding = nn.Embedding.from_pretrained(emb_vecs,
                                                          freeze=False,
                                                          padding_idx=self.vocab.pad())
            embed_dim = self.embedding.embedding_dim
        else:
            embed_dim = config.model.args.embed_dim
            self.embedding = nn.Embedding(len(vocab), embed_dim,
                                          padding_idx=self.vocab.pad())
                                          
        hidden_dim = config.model.args.hidden_dim
        self.encoder = Encoder(input_dim=embed_dim,
                               hidden_dim=hidden_dim)
        self.decoder = AttnDecoder(input_dim=embed_dim,
                                   hidden_dim=hidden_dim,
                                   vocab_size=len(vocab),
                                   use_coverage=config.loss.args.use_coverage)

        # Parameters specific to PGN - Eq. (8)
        self.w_h = nn.Linear(hidden_dim * 2, 1, bias=False)
        self.w_s = nn.Linear(hidden_dim, 1, bias=False)
        self.w_x = nn.Linear(embed_dim, 1, bias=True)

        # Hyper-parameters used during decoding at inference time
        self.beam_size = config.decode.args.beam_size
        self.min_dec_steps = config.decode.args.min_dec_steps
        self.num_return_seq = config.decode.args.num_return_seq


#Make changes in this function


    def load_embeddings(self, which='fasttext'):
        num_oov = 0
        num_in_vocab = 0
        emb_vecs = []
        emb_size = 300
        import fasttext.util
        fasttext.util.download_model('ko', if_exists='ignore')
        ft = fasttext.load_model('cc.ko.300.bin')
        for w in self.vocab._id_to_word:
            if ft.get_word_id(w) == -1:  # out of ft vocab
                w_emb = torch.rand([emb_size, 1])
                nn.init.kaiming_normal_(w_emb, mode='fan_out')
                num_oov += 1
            else:
                w_emb = torch.tensor(ft.get_word_vector(w))
                num_in_vocab += 1
            emb_vecs.append(w_emb)
        emb_vecs = list(map(lambda x: x.squeeze(), emb_vecs))
        emb_vecs = torch.stack(emb_vecs)

        num_total = num_oov + num_in_vocab
        print(f"Loaded embeddings from {which}: {num_in_vocab} out of {num_total} are initialized from {which}")
        return emb_vecs

    def forward(self, enc_input, enc_input_ext, enc_pad_mask, enc_len,
                dec_input, max_oov_len):
        """
        Predict summary using reference summary as decoder inputs (teacher forcing)
        Args:
            enc_input: source text id sequence                      [B x L]
            enc_input_ext: source text id seq w/ extended vocab     [B x L]
            enc_pad_mask: source text padding mask. [PAD] -> True   [B x L]
            enc_len: source text length                             [B]
            dec_input: target text id sequence                      [B x T]
            max_oov_len: max number of oovs in src                  [1]
        Returns:
            final_dists: predicted dist'n using extended vocab      [B x V_x x T]
            attn_dists: attn dist'n from each t                     [B x L x T]
            coverages: coverage vectors from each t                 [B x L x T]
        """

        # Build source text representations from encoder
        enc_emb = self.embedding(enc_input)                         # [B x L x E]
        enc_hidden, (h, c) = self.encoder(enc_emb, enc_len)         # [B x L x 2H]

        # Outputs required for loss computation
        # 1. cross-entropy (negative log-likelihood) loss - Eq. (6)
        final_dists = []

        # 2. coverage loss - Eq. (12)
        attn_dists = []
        coverages = []

        # Initialize decoder inputs
        dec_emb = self.embedding(dec_input)                         # [B x T x E]
        cov = torch.zeros_like(enc_input).float()                   # [B x L]

        for t in range(self.config.data.tgt_max_train):
            input_t = dec_emb[:, t, :]  # Decoder input at this timestep
            vocab_dist, attn_dist, context_vec, h, c = self.decoder(dec_input=input_t,
                                                                    prev_h=h,
                                                                    prev_c=c,
                                                                    enc_hidden=enc_hidden,
                                                                    enc_pad_mask=enc_pad_mask,
                                                                    coverage=cov)
            # Eq. (10) - Compute coverage vector;
            # sum of attn dist over all prev decoder timesteps
            cov = cov + attn_dist

            # Eq. (8) - Compute generation probability p_gen
            context_feat = self.w_h(context_vec)                    # [B x 1]
            decoder_feat = self.w_s(h)                              # [B x 1]
            input_feat = self.w_x(input_t)                          # [B x 1]
            gen_feat = context_feat + decoder_feat + input_feat
            p_gen = torch.sigmoid(gen_feat)                         # [B x 1]

            # Eq. (9) - Compute prob dist'n over extended vocabulary
            vocab_dist = p_gen * vocab_dist                         # [B x V]
            weighted_attn_dist = (1.0 - p_gen) * attn_dist          # [B x L]

            # Concat some zeros to each vocab dist,
            # to hold probs for oov words that appeared in source text
            batch_size = vocab_dist.size(0)
            extra_zeros = torch.zeros((batch_size, max_oov_len),
                                      device=vocab_dist.device)
            extended_vocab_dist = torch.cat([vocab_dist, extra_zeros], dim=-1)  # [B x V_x]

            final_dist = extended_vocab_dist.scatter_add(dim=-1,
                                                         index=enc_input_ext,
                                                         src=weighted_attn_dist)
            # Save outputs for loss computation
            final_dists.append(final_dist)
            attn_dists.append(attn_dist)
            coverages.append(cov)

        final_dists = torch.stack(final_dists, dim=-1)  # [B x V_x x T]
        attn_dists = torch.stack(attn_dists, dim=-1)    # [B x L x T]
        coverages = torch.stack(coverages, dim=-1)      # [B x L x T]

        return {
            'final_dist': final_dists,
            'attn_dist': attn_dists,
            'coverage': coverages
        }

    def inference(self, enc_input, enc_input_ext, enc_pad_mask, enc_len,
                  src_oovs, max_oov_len):
        """
        Predict summary using previous timestep's decoder output as this step's decoder input + beam search
        Args:
            enc_input: source text id sequence                      [B x L]
            enc_input_ext: source text id seq w/ extended vocab     [B x L]
            enc_pad_mask: source text padding mask. [PAD] -> True   [B x L]
            enc_len: source text length                             [B]
            src_oovs: list of source text oovs of each sample       [B]
            max_oov_len: max number of oovs in src                  [1]
        Returns:
            results: dictionary with 'generated_summary'
        """
        # Build source text representation from encoder
        enc_emb = self.embedding(enc_input)  # [B x L x E]
        enc_hidden, (h, c) = self.encoder(enc_emb, enc_len)  # [B x L x 2H]
        #print((h,c).shape)
        # Initialize decoder input
        cov = torch.zeros_like(enc_input).float()  # [B x L]

        # Initialize hypotheses
        batch_size = enc_input.size(0) #[B] should be 1
        # All samples start with a single hypothesis ([START])
        hyps = [
            Hypothesis(tokens=[self.vocab.start()],
                       log_probs=[0.0],
                       hidden_state=h,
                       cell_state=c,
                       coverage=cov)
            for _ in range(batch_size)
        ]
        results = []  # finished hypotheses (those that have emitted the [STOP] token)

        for steps in range(self.config.data.tgt_max_test):
            # Prepare decoder inputs (= previously generated tokens) for this step
            # K : number of hypotheses (we want top-K outputs)
            dec_input = [self.filter_unk(hyp.latest_token) for hyp in hyps] #1 x B
            dec_input = torch.tensor(dec_input,
                                     dtype=torch.long,
                                     device=enc_input.device)                   # [K]  #[B]
            dec_emb = self.embedding(dec_input)                                 # [K x E]
            h = torch.cat([hyp.hidden_state for hyp in hyps], dim=0)            # [1 x H] -> [K x H] #B x H
            c = torch.cat([hyp.cell_state for hyp in hyps], dim=0)              # [1 x H] -> [K x H]
            coverages = torch.cat([hyp.coverage for hyp in hyps], dim=0)        # [1 x L] -> [K x L]
            enc_hiddens = torch.cat([enc_hidden for _ in hyps], dim=0)          # [1 x L x 2H] -> [K x L x 2H]
            enc_pad_masks = torch.cat([enc_pad_mask for _ in hyps], dim=0)

            vocab_dist, attn_dist, context_vec, h, c = self.decoder(dec_input=dec_emb,
                                                                    prev_h=h, prev_c=c,
                                                                    enc_hidden=enc_hiddens,
                                                                    enc_pad_mask=enc_pad_masks,
                                                                    coverage=coverages)

            # Eq. (10) - Compute coverage vector;
            # sum of attn dist over all prev decoder timesteps
            cov = cov + attn_dist

            # Eq. (8) - Compute generation probability p_gen
            context_feat = self.w_h(context_vec)                    # [K x 1]
            decoder_feat = self.w_s(h)                              # [K x 1]
            input_feat = self.w_x(dec_emb)                          # [K x 1]
            gen_feat = context_feat + decoder_feat + input_feat
            p_gen = torch.sigmoid(gen_feat)                         # [K x 1]

            # Eq. (9) - Compute prob dist'n over extended vocabulary
            vocab_dist = p_gen * vocab_dist                         # [K x V]
            weighted_attn_dist = (1.0 - p_gen) * attn_dist          # [K x L]

            # Concat some zeros to each vocab dist,
            # to hold probs for oov words that appeared in source text
            batch_size = vocab_dist.size(0)
            extra_zeros = torch.zeros((batch_size, max_oov_len),
                                      device=vocab_dist.device)
            extended_vocab_dist = torch.cat([vocab_dist, extra_zeros], dim=-1)  # [K x V_x]
            final_dist = extended_vocab_dist.scatter_add(dim=-1,
                                                         index=enc_input_ext,
                                                         src=weighted_attn_dist)

            # Find top-2k most probable token ids and update hypotheses
            log_probs = torch.log(final_dist)
            topk_probs, topk_ids = torch.topk(log_probs,
                                              k=self.beam_size * 2,
                                              dim=-1)

            all_hyps = []
            num_orig_hyps = 1 if steps == 0 else len(hyps)
            for i in range(num_orig_hyps):
                h_i = hyps[i]
                hidden_state_i = h[i].unsqueeze(0)
                cell_state_i = c[i].unsqueeze(0)
                coverage_i = cov[i].unsqueeze(0)

                for j in range(self.beam_size * 2):
                    # Update existing hypothesis with predicted token
                    if topk_ids[i, j].item() == self.vocab.unk():
                        pass
                    else:
                        new_hyp = h_i.extend(token=topk_ids[i, j].item(),
                                             log_prob=log_probs[i, j].item(),
                                             hidden_state=hidden_state_i,
                                             cell_state=cell_state_i,
                                             coverage=coverage_i)
                        all_hyps.append(new_hyp)

            # Find k most probable hypotheses among 2k candidates
            hyps = []
            for h in self.sort_hyps(all_hyps):
                if h.latest_token == self.vocab.stop():
                    if steps >= self.min_dec_steps:
                        results.append(h)
                else:
                    # save for next step
                    hyps.append(h)
                if len(hyps) == self.beam_size or len(results) == self.beam_size:
                    break

            if len(results) == self.beam_size:
                break

        # Reached max decode steps but not enough results
        if len(results) < self.num_return_seq:
            results = results + hyps[:self.num_return_seq - len(results)]

        sorted_results = self.sort_hyps(results)
        best_hyps = sorted_results[:self.num_return_seq]

        # Map token ids to words
        hyp_words = [self.vocab.outputids2words(hyp.tokens, src_oovs[0]) for hyp in best_hyps]

        # Concatenate words to strings
        if self.config.model.use_pretrained and self.config.model.pretrained == 'kobert':
            bpe = True
        else:
            bpe = False
        hyp_results = [postprocess(words, #bpe=bpe,
                                   skip_special_tokens=True,
                                   clean_up_tokenization_spaces=True)
                       for words in hyp_words]
        results = {'generated_summary': hyp_results}
        return results

    def sort_hyps(self, hyps):
        """Sort hypotheses according to their log probability."""
        return sorted(hyps, key=lambda h: h.avg_log_prob, reverse=True)

    def filter_unk(self, idx):
        return idx if idx < self.vocab.size() else self.vocab.unk()

#setup code for training evaluating and testing

In [ ]:
class SummarizationModel(pl.LightningModule):
    def __init__(self, config, vocab):
        super(SummarizationModel, self).__init__()
        self.config = config
        self.vocab = vocab
        self.model = PointerGenerator(config, vocab)
        self.criterion = Loss(args=config.loss.args)
        self.num_step = 0
        self.cov_weight = config.loss.args.cov_weight
        self.rouge = Rouge()
        self.batch_size = config.data_loader.batch_size.train
        
    # when pl.Trainer.fit() is used this method will be called. This will return loss
    # To see in detail go to - https://pytorch-lightning.readthedocs.io/en/stable/common/trainer.html
    def training_step(self, batch, batch_idx):
        # batch class object from data.py has been passed
        output = self.model.forward(enc_input=batch.enc_input,
                                    enc_input_ext=batch.enc_input_ext,
                                    enc_pad_mask=batch.enc_pad_mask,
                                    enc_len=batch.enc_len,
                                    dec_input=batch.dec_input,
                                    max_oov_len=batch.max_oov_len)

        nll_loss, cov_loss = self.criterion(output=output,
                                            batch=batch)
        loss = nll_loss + self.cov_weight * cov_loss
        # self.log('train_loss', loss, on_step=True, on_epoch=False, prog_bar=True, logger=True)
        self.logger.log_metrics({'train_loss': loss,
                                 'train/nll_loss': nll_loss}, self.num_step)
        if self.cov_weight > 0:
            self.logger.log_metrics({'train/cov_loss': cov_loss}, self.num_step)
        self.num_step += 1
        return loss

    def validation_step(self, batch, batch_idx):
        output = self.model.forward(enc_input=batch.enc_input,
                                    enc_input_ext=batch.enc_input_ext,
                                    enc_pad_mask=batch.enc_pad_mask,
                                    enc_len=batch.enc_len,
                                    dec_input=batch.dec_input,
                                    max_oov_len=batch.max_oov_len)
        nll_loss, cov_loss = self.criterion(output=output,
                                            batch=batch)
        loss = nll_loss + self.cov_weight * cov_loss
        

        # result = self.test_step(batch, batch_idx)
        # scores = self.rouge.get_scores(result['generated_summary'],
        #                                result['gold_summary'], avg=True)
        # rouge_1 = scores['rouge-1']['f'] * 100.0
        # rouge_2 = scores['rouge-2']['f'] * 100.0
        # rouge_l = scores['rouge-l']['f'] * 100.0
        pred = {
        #     'rouge_1': rouge_1,
        #     'rouge_2': rouge_2,
        #     'rouge_l': rouge_l,
             'val_loss': loss
         }
        #val_loss = []
        #for p in pred:
        #  val_loss.append(p['val_loss'])
          #val_loss.append(pred[p])
        #val_loss_avg = sum(val_loss) / len(val_loss)
        self.log('val_loss', loss, on_step=True, on_epoch=False, prog_bar=False, logger=True)
        self.logger.log_metrics({'val_loss': loss}, self.num_step)
        
        return pred

    def validation_epoch_end(self, validation_step_outputs):
      #  rouge_1, rouge_2, rouge_l= [], [], [] 
        val_loss = [] 
        for pred in validation_step_outputs:
  #         rouge_1.append(pred['rouge_1'])
  #         rouge_2.append(pred['rouge_2'])
  #         rouge_l.append(pred['rouge_l'])
            val_loss.append(pred['val_loss'])
  #     rouge_1_avg = sum(rouge_1) / len(rouge_1)
  #     rouge_2_avg = sum(rouge_2) / len(rouge_2)
  #     rouge_l_avg = sum(rouge_l) / len(rouge_l)
        val_loss_avg = sum(val_loss) / len(val_loss)
        results = {
  #         'rouge_1_avg': rouge_1_avg,
  #         'rouge_2_avg': rouge_2_avg,
  #         'rouge_l_avg': rouge_l_avg,
            'val_loss_avg': val_loss_avg,
        }
        self.log_dict(results)
        return results

    def test_step(self, batch, batch_idx):
        result = self.model.inference(enc_input=batch.enc_input,
                                      enc_input_ext=batch.enc_input_ext,
                                      enc_pad_mask=batch.enc_pad_mask,
                                      enc_len=batch.enc_len,
                                      src_oovs=batch.src_oovs,
                                      max_oov_len=batch.max_oov_len)
        result['source'] = [''.join(w) for w in batch.src_text]
        result['gold_summary'] = [''.join(w) for w in batch.tgt_text]
        return result

    def predict_step(self, batch, batch_idx, dataloader_idx: int = None):
        generated_summary = [] 
        source = []
        gold_summary = []
        result = self.model.inference(enc_input=batch.enc_input,
                                      enc_input_ext=batch.enc_input_ext,
                                      enc_pad_mask=batch.enc_pad_mask,
                                      enc_len=batch.enc_len,
                                      src_oovs=batch.src_oovs,
                                      max_oov_len=batch.max_oov_len)

        result['source'] = [''.join(w) for w in batch.src_text]
        result['gold_summary'] = [''.join(w) for w in batch.tgt_text]
        return result

    def configure_optimizers(self):
        args = self.config.optimizer.args
        lr = args.lr
        lr_init_accum = args.lr_init_accum
        params = self.parameters()
        optimizer = Adagrad(
            params=params,
            lr=lr,
            initial_accumulator_value=lr_init_accum,
        )
        return optimizer

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping


#Train code

In [ ]:
# config.data_loader.batch_size.train = 4096

In [ ]:
import argparse
import os
import sys

import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers



def main(config):
    # fix random seeds for reproducibility
    SEED = 123
    pl.seed_everything(SEED)

    # generate experiment name
    exp_name = generate_exp_name(config)
    
    # Train
    # 1. Load dataset, build or load vocab file
    # if use pre-built vocab
    # if os.path.exists('/content/drive/MyDrive/ko-pointer-generator/logs/vocab.json') and config.load_vocab:
    if os.path.exists('/content/drive/MyDrive/ko-pointer-generator/logs/vocab.json'):
        load_vocab = '/content/drive/MyDrive/ko-pointer-generator/logs/vocab.json'
    else:
        load_vocab = None

    # 1-1. Training dataset
    # If validation set is not specified, take 10% of training data as validation set
    # split_dev = 0.1 if config.path.val == '' else 0.0
    # (train_data, val_data), vocab = build_dataset(
    #     data_path=config.path.train,
    #     load_vocab=load_vocab,
    #     config=config,
    #     is_train=True,
    #     split_dev=split_dev,
    # )
    
    train_data, vocab = build_dataset(
        data_path=config.path.train,
        load_vocab=load_vocab,
        config=config,
        is_train=True
    )

    # save vocab file
    vocab.save(os.path.join(f'/content/drive/MyDrive/ko-pointer-generator/logs/{exp_name}', config.path.vocab))

    print(lambda data, v=vocab, t=config.data.tgt_max_train: Batch(data=data, vocab=v,max_decode=t))

    train_loader = build_dataloader(
        dataset=train_data,
        vocab=vocab,
        batch_size=config.data_loader.batch_size.train,
        max_decode=config.data.tgt_max_train,
        is_train=True,
        num_workers=config.data_loader.num_workers,
    )
    
    # 1-2. Load validation dataset
    # For validation, we use NIKL
    val_data = None
    if val_data is None:
        val_data = build_dataset(
            data_path=config.path.val,
            config=config,
            is_train=False,
            vocab=vocab
        )

    val_loader = build_dataloader(
        dataset=val_data,
        vocab=vocab,
        batch_size=config.data_loader.batch_size.val,
        max_decode=config.data.tgt_max_test,
        is_train=False,
        num_workers=config.data_loader.num_workers,
    )

    # 2. Build model instance
    model = SummarizationModel(
        config=config,
        vocab=vocab
    )

    # 3. Set logger, trainer
    tb_logger = pl_loggers.TensorBoardLogger(
        save_dir='/content/drive/MyDrive/ko-pointer-generator/logs/',
        name=exp_name,
    )
    tb_logger.log_hyperparams(config)

    
    monitor = 'val_loss_avg'
    mode = 'min'
   
    # filepath = '{epoch}-{' + monitor + ':.2f}'
    # checkpoint_callback = ModelCheckpoint(
    #     dirpath =f'/content/drive/MyDrive/ko-pointer-generator/logs/{exp_name}',
    #     filename = filepath,
    #     verbose=False,
    #     monitor=monitor,
    #     mode=mode,
    #     save_top_k=5,
    # )

    # early stopping
    early_stop_callback = EarlyStopping(
        monitor=monitor,
        min_delta=0.00,
        patience=10,
        verbose=True,
        mode=mode,
        strict=False
    )

    if config.device == -1:
        gpus = None
    else:
        gpus = [config.device]
        
    trainer = pl.Trainer(
        logger=tb_logger,
        callbacks=early_stop_callback,
        gpus=gpus,
        resume_from_checkpoint=config.model_path,
        max_epochs=config.trainer.epochs,
        gradient_clip_val=config.trainer.max_grad_norm,
        log_every_n_steps=500,
        auto_scale_batch_size=False,
    )

    # 4. Train!
    trainer.fit(model, train_loader, val_loader)

    # 5. Evaluation
    test_data = build_dataset(
        data_path=config.path.test,
        config=config,
        is_train=False,
        vocab=vocab
    )

    test_loader = build_dataloader(
        dataset=test_data,
        vocab=vocab,
        batch_size=1,
        max_decode=config.data.tgt_max_test,
        is_train=False,
        num_workers=config.data_loader.num_workers,
    )

    test_outputs = trainer.predict(model, test_loader)
    print(test_outputs)
    print(test_outputs[0])
    print(len(test_outputs[0]))
    output_name = generate_output_name(config)
    write_output(test_loader=test_loader,
                 test_outputs=test_outputs,
                 fname=os.path.join(f'/content/drive/MyDrive/ko-pointer-generator/logs/{exp_name}', output_name))


if __name__ == "__main__":
    args = {}
    args['config_path'] =  '/content/drive/MyDrive/ko-pointer-generator/config.json'
    args['load_vocab'] = True
    args['stop_with'] = 'loss'
    args['device'] = 0
    args['mds'] = None
    args['model_path'] = None 
    args['exp_name'] = ''
    args['note']='' 
    '''
    args = argparse.ArgumentParser(description='Pointer-generator network')

    args.add_argument(
        '-cp', '--config-path',
        default='/content/drive/MyDrive/ko-pointer-generator/config.json',
        type=str,
        help='path to config file'
    )


    args.add_argument(
        '--load-vocab',
        action='store_true',
        default=False,
        help='whether to load pre-built vocab file'
    )

    args.add_argument(
        '--stop-with',
        default='rl',
        type=str,
        choices=['loss', 'r1', 'r2', 'rl'],
        help='validation evaluation metric to perform early stopping'
    )


    args.add_argument(
        '-d', '--device',
        default=-1,
        type=int,
        help='gpu device number to use. if cpu, set this argument to -1'
    )


    # sys.path.append(
    #     os.path.dirname(os.path.abspath(os.path.dirname("__file__")))
    # )
    '''
    config = config_parser(args)
    main(config)

INFO:lightning_lite.utilities.seed:Global seed set to 123


Experiment results saved in logs/11-28-19:59:00
Loading dataset from /content/drive/MyDrive/ko-pointer-generator/cnn_data/cnn_training_set.csv
["'luke", "'legend", "'leading", "'interesting", "'humiliated", "'hitler", "'gutted", "'eight", "'economic", "'building", "'beat", "'12", '£48,000', '£1bn', '£185', '£175,000', '£145', 'zarooni', 'yettaw', 'xxxx', 'x5', 'world-leading', 'withstanding', 'warmup', 'walmsley', 'wail', 'vreeland', 'vouch', 'vma', 'verve', 'vegans', 'varun', 'vandalising', 'vaisey', 'usaf', 'urquhart', 'urbina', 'unyielding', 'unsympathetic', 'unshaven', 'undercutting', 'uncomplicated', 'ul', 'tynecastle', 'two-years-old', 'turlington', 'tui', 'triggs', 'tri-state', 'transsexuals', 'transpires', 'tomtom', 'timetables', 'thrusts', 'thorp', 'thirty-five', 'thigh-high', 'thicken', 'then.', 'thawed', 'tay', 'tattersall', 'taming', 'tak', 'swaddling', 'storytellers', 'stortford', 'stinky', 'staphylococcus', 'stage.', 'sprinkles', 'speedily', 'southmead', 'soluble', 'sneha

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=[0])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[0])` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | model     | PointerGenerator | 21.3 M
1 | criterion | Loss             | 0     
---------------

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Loading dataset from /content/drive/MyDrive/ko-pointer-generator/cnn_data/cnn_test_set.csv


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 4487it [00:00, ?it/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



TypeError: ignored

In [ ]:
args = {}
args['config_path'] =  '/content/drive/MyDrive/ko-pointer-generator/config.json'
args['load_vocab'] = False
args['stop_with'] = 'rl'
args['device'] = 0
args['mds'] = None
args['model_path'] = None 
args['exp_name'] = ''
args['note']='' 
config = config_parser(args)
'''
    args = argparse.ArgumentParser(description='Pointer-generator network')

    args.add_argument(
        '-cp', '--config-path',
        default='/content/drive/MyDrive/ko-pointer-generator/config.json',
        type=str,
        help='path to config file'
    )


    args.add_argument(
        '--load-vocab',
        action='store_true',
        default=False,
        help='whether to load pre-built vocab file'
    )

    args.add_argument(
        '--stop-with',
        default='rl',
        type=str,
        choices=['loss', 'r1', 'r2', 'rl'],
        help='validation evaluation metric to perform early stopping'
    )


    args.add_argument(
        '-d', '--device',
        default=-1,
        type=int,
        help='gpu device number to use. if cpu, set this argument to -1'
    )


    # sys.path.append(
    #     os.path.dirname(os.path.abspath(os.path.dirname("__file__")))
    # )
'''
  
    

In [ ]:

def build_dataloader(dataset, vocab, batch_size, max_decode, is_train, num_workers):
    shuffle = True if is_train else False
    data_loader = DataLoader(dataset,
                             batch_size=batch_size,
                             shuffle=shuffle,
                             collate_fn=lambda data, v=vocab, t=max_decode: Batch(data=data,
                                                                                  vocab=v,
                                                                                  max_decode=t),
                             num_workers=num_workers)
    return data_loader

In [ ]:
#exp_name = generate_exp_name(config)
import os   
    # Train
    # 1. Load dataset, build or load vocab file
    # if use pre-built vocab
    # if os.path.exists('/content/drive/MyDrive/ko-pointer-generator/logs/vocab.json') and config.load_vocab:
if os.path.exists('/content/drive/MyDrive/ko-pointer-generator/logs/11-20-22:19:51/vocab.json'):
    load_vocab = '/content/drive/MyDrive/ko-pointer-generator/logs/11-20-22:19:51/vocab.json'
else:
    load_vocab = None

    # 1-1. Training dataset
    # If validation set is not specified, take 10% of training data as validation set
    # split_dev = 0.1 if config.path.val == '' else 0.0
    # (train_data, val_data), vocab = build_dataset(
    #     data_path=config.path.train,
    #     load_vocab=load_vocab,
    #     config=config,
    #     is_train=True,
    #     split_dev=split_dev,
    # )
train_data, vocab = build_dataset(
        data_path=config.path.train,
        load_vocab=load_vocab,
        config=config,
        is_train=True
    )

    # save vocab file
vocab.save(os.path.join(f'/content/drive/MyDrive/ko-pointer-generator/logs/test', config.path.vocab))

print(lambda data, v=vocab, t=config.data.tgt_max_train: Batch(data=data, vocab=v,max_decode=t))

train_loader = build_dataloader(
        dataset=train_data,
        vocab=vocab,
        batch_size=config.data_loader.batch_size.train,
        max_decode=config.data.tgt_max_train,
        is_train=True,
        num_workers=config.data_loader.num_workers,
    )
    